# Show how MDS CPU load has changed over time

Uses TOKIO to pull the MDS CPU load over a period of time, calculates the average load over each day, and then plots the resulting daily averages over time.

In [ ]:
%matplotlib inline

In [ ]:
import datetime

In [ ]:
import matplotlib
matplotlib.rcParams['font.size'] = 16
import pandas

In [ ]:
import tokio

In [ ]:
# date_start = datetime.datetime(2018, 1, 1)
date_start = datetime.datetime(2019, 1, 1)
date_end = datetime.datetime(2019, 5, 1)

date_now = date_start

In [ ]:
avg_cpu = []
while date_now < date_end:
    df = tokio.tools.hdf5.get_dataframe_from_time_range(
        fsname='cscratch',
        dataset_name='mdservers/cpuload',
        datetime_start=date_now,
        datetime_end=date_now + datetime.timedelta(days=1),
    )
    if df is not None:
        mds_cpu = df.iloc[:, 0].dropna().mean()
        avg_cpu.append({
            'date': date_now,
            'mds_cpu': mds_cpu,
        })
        print(avg_cpu[-1])
    date_now = date_now + datetime.timedelta(days=1)

In [ ]:
df = pandas.DataFrame.from_records(avg_cpu, index='date')

In [ ]:
df.plot()

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(8, 4))

# Plot raw data points
ax.plot(df, linestyle='', marker='o', alpha=0.5, label="Daily mean")

# Plot sliding seven-day average
ax.plot(df.resample('7D').mean(), color='C3', linewidth=2, label="7-day mean")

ax.grid()
ax.set_ylabel("Average MDS\nCPU Load (%)")
ax.set_title("MDS0 Average CPU Load")
ax.legend()
for label in ax.get_xmajorticklabels():
    label.set_rotation(30)
    label.set_horizontalalignment("right")